In [7]:
import numpy as np
class Node():

    def __init__(self, identifier, parent=None, position=None, operation=None):
        self.identifier = identifier
        self.parent = parent
        self.position = position
        self.children = list()
        self.operation = operation

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def add_child(self, child):
        self.children.append(child)
       

    def read_from_file(file_name):
        file_handle = open(file_name)
        map = file_handle.readlines()
        file1 = []
        for item in map:
             file1.append(item.strip('\n'))
        n = int(file1.pop(0))
 
        return file1
    
def graphsearch(maze, start, end):
  
    # Create start and end node
    node_id = 0
    start_node = Node('N'+str(node_id), None, start, 'S')
    print(f"Start Node: {start_node.identifier}, pos: {start_node.position}, cost: {start_node.g}, operation: {start_node.operation}")
    node_id += 1
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node('G', None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you reach the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Remove current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # If goal is reached
        if current_node == end_node:
            path = ['G']
            current = current_node
            cost = current_node.g
            while current is not None:
                path.append(current.operation)
                current = current.parent
            
            return path[::-1], True, cost # Return reversed path

        # Left, right, up, down movements
        for new_position in [(0, -1, 'L'), (0, 1, 'R'), (-1, 0, 'U'), (1, 0, 'D')]: # relative positions

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within the boundaries
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # ignore parent
            if current_node.parent is not None and node_position == current_node.parent.position:
                continue

            # Create new node
            new_node = Node('N'+str(node_id), current_node, node_position, new_position[2])
            node_id += 1

            # Append
            current_node.add_child(new_node)

            # Create the f, g, and h values
            new_node.g = current_node.g + 2
            new_node.h = (abs(new_node.position[0] - end_node.position[0])) + (abs(new_node.position[1] - end_node.position[1]))
            new_node.f = new_node.g + new_node.h
            print(f"Child Node: {new_node.identifier}, pos: {new_node.position}, cost: {new_node.g}, operation: {new_node.operation}")

            # Child is already in the open list
            for open_node in open_list:
                if new_node == open_node and new_node.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(new_node)

                #The same procedure is carried out for the 4 diagonal positions
        for new_position in [(-1, -1, 'LU'), (-1, 1, 'RU'), (1, -1, 'LD'), (1, 1, 'RD')]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            adj_node1 = (current_node.position[0], current_node.position[1] + new_position[1])
            adj_node2 = (current_node.position[0] + new_position[0], current_node.position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Make sure walkable terrain due to adjacent mountains
            if maze[adj_node1[0]][adj_node1[1]] != 0 or maze[adj_node2[0]][adj_node2[1]] != 0:
                continue

            # ignore parent
            if current_node.parent is not None and node_position == current_node.parent.position:
                continue

            # Create new node
            new_node = Node('N'+str(node_id), current_node, node_position, new_position[2])
            node_id += 1

            # Child is on the closed list
            for closed_child in closed_list:
                if new_node == closed_child:
                    continue
            
            # Append
            current_node.add_child(new_node)

            # Create the f, g, and h values
            new_node.g = current_node.g + 1
            new_node.h = (abs(new_node.position[0] - end_node.position[0])) + (abs(new_node.position[1] - end_node.position[1]))
            new_node.f = new_node.g + new_node.h
            print(f"Child Node: {new_node.identifier}, pos: {new_node.position}, cost: {new_node.g}, operation: {new_node.operation}")

            # Child is already in the open list
            for open_node in open_list:
                if new_node == open_node and new_node.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(new_node)

    return list(), False, -1



def main():
    file=read_from_file("input1.txt")
    for i in range(len(file)):
        for j in range(len(file)):
            if (file[i][j]=='S'):
                start=(i,j)
            elif (file[i][j]=='G'):
                end=(i,j)
    f=np.zeros((len(file),len(file)))
    for i in range(len(file)):
        for j in range(len(file)):
            if (file[i][j]=='S' or file[i][j]=='R' or file[i][j]=='G'):
                f[i][j]=0
            elif(file[i][j]=='X'):
                f[i][j]=1
   
    path, path_exists, cost = graphsearch(f, start, end)
    if path_exists:
        print(f"path: {'-'.join(path)}")
        print(f"cost: {cost}")
    else:
        print('NO-PATH')

if __name__ == '__main__':
   
    main()



Start Node: N0, pos: (0, 0), cost: 0, operation: S
Child Node: N1, pos: (0, 1), cost: 2, operation: R
Child Node: N2, pos: (1, 0), cost: 2, operation: D
Child Node: N3, pos: (1, 1), cost: 1, operation: RD
Child Node: N4, pos: (1, 0), cost: 3, operation: L
Child Node: N5, pos: (0, 1), cost: 3, operation: U
Child Node: N6, pos: (2, 1), cost: 3, operation: D
Child Node: N7, pos: (2, 0), cost: 2, operation: LD
Child Node: N8, pos: (2, 0), cost: 5, operation: L
Child Node: N9, pos: (2, 2), cost: 5, operation: R
Child Node: N10, pos: (1, 0), cost: 4, operation: LU
Child Node: N11, pos: (2, 1), cost: 4, operation: R
Child Node: N12, pos: (1, 0), cost: 4, operation: U
Child Node: N13, pos: (0, 2), cost: 4, operation: R
Child Node: N14, pos: (1, 1), cost: 4, operation: D
Child Node: N15, pos: (1, 0), cost: 3, operation: LD
Child Node: N16, pos: (1, 1), cost: 4, operation: R
Child Node: N17, pos: (2, 0), cost: 4, operation: D
Child Node: N18, pos: (0, 1), cost: 3, operation: RU
Child Node: N19, 